In [1]:
import time, array, random, copy, math
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['text.latex.preamble'] ='\\usepackage{libertine}\n\\usepackage[utf8]{inputenc}'

import seaborn
seaborn.set(style='whitegrid')
seaborn.set_context('notebook')

In [3]:
from deap import algorithms, base, benchmarks, tools, creator

In [4]:
random.seed(a=81)

In [5]:
data = pd.read_excel('VD-LVTN.xlsx', 3, None)

In [6]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,x,x,0,3,15,90,4,15,95,x,x,x,x,x,x
1,1,0,FS,0,5,135,87,6,135,92,x,x,x,x,x,x
2,2,1,FS,0,50,x,x,x,x,x,x,x,x,x,x,x
3,3,0,FS,0,7,98,85,9,90,80,10,85,85,x,x,x
4,4,3,FS,0,19,612,80,23,550,85,25,581,88,x,x,x
5,5,4,FS,0,42,447,85,46,515,82,50,481,85,x,x,x
6,6,5,FS,0,7,52,89,9,45,75,10,48.5,78,x,x,x
7,7,"6,2","FS,FS","0,0",35,5780,85,40,5690,89,43,5735,92,30,5910,75
8,8,7,FS,0,47,1285,95,52,1240,85,55,1262.5,88,x,x,x
9,9,8,FF,0,11,212,85,14,200,77,16,206,80,x,x,x


In [7]:
contraints = list(data[[1,2,3]].values)

In [8]:
adj_list = defaultdict(lambda:[])
parent_list = defaultdict(lambda:[])
for idx, (u, v, z) in enumerate(contraints):
    if u == 'x':
        continue
    adj_vert = [int(x) for x in str(u).split(',')]
    adj_constr = str(v).split(',')
    adj_delay = [int(x) for x in str(z).split(',')]
    for i, j, k in zip(adj_vert, adj_constr, adj_delay):
        adj_list[i] += [(idx, j, k)]
        parent_list[idx] += [(i, j, k)]

In [9]:
for i in range(len(contraints)):
    if not adj_list[i]:
        adj_list[i] += [('F', 'FS', 0)]
        parent_list['F'] += [(i, 'FS', 0)]
    if not parent_list[i]:
        adj_list['S'] += [(i, 'FS', 0)]
        parent_list[i] += [('S', 'FS', 0)]

In [42]:
a = DiGraph(adj_list, parent_list, data[4].values)
a.forward_pass()
print(a.get_total_duration())

181


In [40]:
class DiGraph:

    def __init__(self, adj_list: dict, parent_list: dict, duration: list):
        self.__adj_list = adj_list
        self.__parent_list = parent_list
        self.__cpm = defaultdict(lambda:defaultdict(lambda:0))
        self.__duration = defaultdict(lambda:0)
        self.set_duration(duration)

    def forward_pass(self):
        q = []
        q.append('S')
        while q:
            u = q.pop(0)
            self.__cpm[u]['EF'] = self.__cpm[u]['ES'] + self.__duration[u]
            for v, rela, delay in self.__adj_list[u]:
                if rela == 'FF':
                    self.__cpm[v]['ES'] = max(self.__cpm[v]['ES'], self.__cpm[u]['EF'] - self.__duration[v] + delay)
                elif rela == 'FS':
                    self.__cpm[v]['ES'] = max(self.__cpm[v]['ES'], self.__cpm[u]['EF'] + delay)
                q.append(v)
    
    def get_cpm(self):
        return self.__cpm

    def set_duration(self, duration):
        for i, val in enumerate(duration):
            self.__duration[i] = val

    def get_total_duration(self):
        return self.__cpm['F']['EF']